In [2]:
import json
from os import path as ospath
import pandas as pd
import numpy as np
import random
import pprint
from src.utils import write_to_json
terms_fp = './downloads/input/full_go_annotated.json'
annotations_fp = './downloads/input/human_iba_annotations.json'
sample_annotations_fp = './data/sample_human_iba_annotations.json'
articles_fp = './downloads/clean-articles.json'
gene_info_fp = './downloads/input/human_iba_gene_info.json'
taxon_fp = './downloads/input/taxon_lkp.json'
clean_annotations_fp = './downloads/human_iba_annotations_clean.json'
filtered_terms_fp = './data/filtered_terms.json'
filtered_slim_terms_fp = './data/filtered_slim_terms.json'
filtered_refs_fp = './data/filtered_refs.json'
filtered_evidences_fp = './data/filtered_evidences.json'
duplicate_gene_fp = './downloads/duplicate_gene.csv'
duplicate_symbol_fp = './downloads/duplicate_symbol.csv'

In [3]:
def get_pd_row(df, k):
    return dict(df.loc[k].dropna())

def spread_terms(df, terms):
    return [get_pd_row(df, term) for term in terms]

def get_pd_row_key(df, k):
    try:
        return dict(df.loc[k].dropna())
    except KeyError:
        return None

def get_aspect(df, k):
    row = get_pd_row_key(df, k)
    if row is None:
        return 'no' + str(k)
    aspect = row.get('aspect', None)
    return aspect

def get_evidence(df, genes_df, row):
    result = []
    for evidence in row['evidence']:
        result_ref =[]
        for reference in evidence['references']:
            result_ref.append(get_pd_row_key(df, reference))
        gene_row = get_pd_row_key(genes_df, evidence['with_gene_id'])
       
        evidence_item = {
          'with_gene_id':gene_row,
          'group': evidence['group'],
          'references':result_ref
        }
        result.append(evidence_item)
            
    return result

def get_evidence_type(row):
    for evidence in row['evidence']:
        if evidence["with_gene_id"]['gene'] == row['gene']:
            return 'direct'
        if len(evidence['references']) == 0:
            return 'n/a' 
            
    return 'homology'



def count_unique_refs_row(evidences):
    refs = set()
        
    for evidence in evidences:
        for ref in evidence['references']:
            refs.add(ref)
            
    return len(list(refs))

def get_other(row):
    if len(row['slim_terms'])==0:
        return ASPECT_OTHER_MAP[row['term']['apsect']]
    
    
def count_unique_slims(evidences):
    refs = set()
        
    for evidence in evidences:
        for ref in evidence['references']:
            refs.add(ref)
            
    return len(list(refs))
            
    


In [4]:
def count_unique_refs(df):
    refs = set()
        
    for evidences in list(df['evidence']):
        for evidence in evidences:
            for ref in evidence['references']:
                refs.add(ref)
            
    return list(refs)

count_uniq = count_unique_refs()
write_to_json(count_uniq, filtered_refs_fp)
print(len(count_uniq))

TypeError: count_unique_refs() missing 1 required positional argument: 'df'

In [5]:
%%time
annos_df = pd.read_json(annotations_fp)
#annos_df = annos_df.loc[2000:2010];
annos_df['ref_count'] = annos_df['evidence'].apply(lambda x: count_unique_refs_row(x))
annos_df = annos_df.sort_values(by='ref_count')

annos_df

CPU times: total: 1.64 s
Wall time: 2.77 s


,gene,gene_symbol,gene_name,term,slim_terms,qualifier,evidence,group,evidence_type,ref_count
90905,UniProtKB:X6R8D5,,Uncharacterized protein,UNKNOWN:0003,[UNKNOWN:0003],None,[],GO_Central,NaN,0
73378,UniProtKB:P62910,RPL32,60S ribosomal protein L32,UNKNOWN:0002,[UNKNOWN:0002],None,[],GO_Central,NaN,0
73380,UniProtKB:Q96Q05,TRAPPC9,Trafficking protein particle complex subunit 9,UNKNOWN:0001,[UNKNOWN:0001],None,[],GO_Central,NaN,0
73381,UniProtKB:Q96Q05,TRAPPC9,Trafficking protein particle complex subunit 9,UNKNOWN:0002,[UNKNOWN:0002],None,[],GO_Central,NaN,0
16989,UniProtKB:Q86YV9,HPS6,Hermansky-Pudlak syndrome 6 protein,UNKNOWN:0001,[UNKNOWN:0001],None,[],GO_Central,NaN,0
...,...,...,...,...,...,...,...,...,...,...
3532,UniProtKB:Q16620,NTRK2,BDNF/NT-3 growth factors receptor,GO:0007275,[GO:0048856],None,"[{'with_gene_id': 'RGD:2561', 'references': ['...",GO_Central,homology,546
6951,UniProtKB:Q5JZY3,EPHA10,Ephrin type-A receptor 10,GO:0007275,[GO:0048856],None,"[{'with_gene_id': 'ZFIN:ZDB-GENE-010126-2', 'r...",GO_Central,homology,546
941,UniProtKB:P35916,FLT4,Vascular endothelial growth factor receptor 3,GO:0007275,[GO:0048856],None,"[{'with_gene_id': 'UniProtKB:P35916', 'referen...",GO_Central,direct,546
17780,UniProtKB:P29323,EPHB2,Ephrin type-B receptor 2,GO:0007275,[GO:0048856],None,"[{'with_gene_id': 'MGI:MGI:104771', 'reference...",GO_Central,homology,546


In [7]:
annos_df2 = annos_df.head(100);
count_uniq = count_unique_refs(annos_df2)
print(len(count_uniq))

anno_json = annos_df2.to_json(orient="records", default_handler=None)
json_str = json.loads(anno_json)

write_to_json(json_str, ospath.join('.', sample_annotations_fp))

0


In [3]:
terms_df = pd.read_json(terms_fp, dtype={'is_goslim':bool})
terms_df = terms_df.set_index('ID', drop=False)
terms_df = terms_df.rename(columns={'ID': 'id', 'LABEL': 'label', 'hasOBONamespace':'aspect'})
terms_df['aspect'] = terms_df['aspect'].str.replace('_', ' ')


#write_to_json(json_str, ospath.join('out.json'))


In [8]:
articles_df = pd.read_json(articles_fp)
articles_df = articles_df.set_index('pmid', drop=False)

,pmid,title,date,authors
pmid,,,,
PMID:10499580,PMID:10499580,The novel Cer-like protein Caronte mediates th...,1999 Sep 16,"[Rodríguez Esteban C, Capdevila J, Economides ..."
PMID:18282108,PMID:18282108,Subfunctionalization of duplicated zebrafish p...,2008 Feb,"[Kleinjan DA, Bancewicz RM, Gautier P, Dahm R,..."
PMID:1426252,PMID:1426252,Biochemical properties of rat protein kinase C...,1992 Nov 9,"[Dekker LV, Parker PJ, McIntyre P]"
PMID:19501188,PMID:19501188,Subcellular localization of ceramide kinase an...,2009 Oct,"[Rovina P, Schanzer A, Graf C, Mechtcheriakova..."
PMID:8810243,PMID:8810243,Promotion of mitochondrial membrane complex as...,1996 Oct 4,"[Rep M, van Dijl JM, Suda K, Schatz G, Grivell..."
...,...,...,...,...
PMID:15905169,PMID:15905169,ICln159 folds into a pleckstrin homology domai...,2005 Sep 2,"[Fürst J, Schedlbauer A, Gandini R, Garavaglia..."
PMID:22851168,PMID:22851168,Glycosphingolipids are essential for intestina...,2012 Sep 21,"[Jennemann R, Kaden S, Sandhoff R, Nordström V..."
PMID:11104725,PMID:11104725,Characterization of an axonemal dynein heavy c...,2000 Dec,"[Reed W, Carson JL, Moats-Staats BM, Lucier T,..."


In [9]:
taxon_df = pd.read_json(taxon_fp, dtype={'taxon_id':str})
#taxon_df

In [16]:
genes_df = pd.read_json(gene_info_fp, dtype={'taxon_id':str})
genes_df['taxon_id'] = genes_df['taxon_id'].str.replace('taxon:', '')
genes_df = genes_df.merge(taxon_df, how='left', on='taxon_id')
genes_df = genes_df.set_index('gene', drop=False)
#genes_df = genes_df['coordinates'].apply(lambda x: x).explode()
#genes_df['taxon_abbr'].unique()
genes_df

,gene,gene_symbol,gene_name,taxon_id,panther_family,long_id,coordinates_chr_num,coordinates_start,coordinates_end,coordinates_strand,taxon_label,taxon_abbr
gene,,,,,,,,,,,,
UniProtKB:P17535,UniProtKB:P17535,JUND,Transcription factor jun-D,9606,PTHR11462,HUMAN|HGNC=6206|UniProtKB=P17535,19,18279760.0,18280929.0,-1.0,Homo sapiens,Hsa
UniProtKB:P05412,UniProtKB:P05412,JUN,Transcription factor AP-1,9606,PTHR11462,HUMAN|HGNC=6204|UniProtKB=P05412,1,58780788.0,58784327.0,-1.0,Homo sapiens,Hsa
UniProtKB:Q92973,UniProtKB:Q92973,TNPO1,Transportin-1,9606,PTHR10527,HUMAN|HGNC=6401|UniProtKB=Q92973,5,72893676.0,72905429.0,1.0,Homo sapiens,Hsa
TAIR:locus:2168586,TAIR:locus:2168586,KPNB1,Importin subunit beta-1,3702,None,ARATH|TAIR=locus=2168586|UniProtKB=Q9FJD4,NaN,NaN,NaN,NaN,Arabidopsis thaliana,Ath
UniProtKB:Q03052,UniProtKB:Q03052,POU3F1,"POU domain, class 3, transcription factor 1",9606,PTHR11636,HUMAN|HGNC=9214|UniProtKB=Q03052,1,38043827.0,38046794.0,-1.0,Homo sapiens,Hsa
...,...,...,...,...,...,...,...,...,...,...,...,...
UniProtKB:Q9H1M4,UniProtKB:Q9H1M4,DEFB127,Beta-defensin 127,9606,None,HUMAN|HGNC=16206|UniProtKB=Q9H1M4,20,157470.0,159163.0,1.0,Homo sapiens,Hsa
UniProtKB:Q9HAC7,UniProtKB:Q9HAC7,SUGCT,Succinate--hydroxymethylglutarate CoA-transferase,9606,None,HUMAN|HGNC=16001|UniProtKB=Q9HAC7,7,40135016.0,40860631.0,1.0,Homo sapiens,Hsa
UniProtKB:Q9NPH9,UniProtKB:Q9NPH9,IL26,Interleukin-26,9606,None,HUMAN|HGNC=17119|UniProtKB=Q9NPH9,12,68201351.0,68225821.0,-1.0,Homo sapiens,Hsa


In [ ]:
annos_df = pd.read_json(annotations_fp)[:1000]
anno_json = annos_df.to_json(orient="records", default_handler=None)
json_str = json.loads(anno_json)
write_to_json(json_str, ospath.join(sample_annotations_fp))
#pprint.pprint(json_str)

In [5]:
annos_df

NameError: name 'annos_df' is not defined

In [ ]:
%%time

annos_df = pd.read_json(annotations_fp)[:100]
annos_df = annos_df.merge(genes_df[['taxon_id', 'taxon_label',
                                    'taxon_abbr', 
                                    'coordinates_chr_num','coordinates_start','coordinates_end','coordinates_strand']], how='left', left_on="gene", right_index=True)
#annos_df['aspect'] = annos_df['term'].apply(lambda x: get_pd_row(terms_df, x)['aspect'])
annos_df



In [17]:
%%time

def count_evidence(evidences):
    return len(evidences)

def get_groups(evidences):
    groups = set()
    for evidence in evidences:
        groups.add(evidence['group'])
    return list(groups)
    
annos_df = pd.read_json(annotations_fp)
annos_df = annos_df.merge(genes_df[
    ['taxon_id', 
     'taxon_label', 
     'taxon_abbr' , 
     'coordinates_chr_num',
     'coordinates_start',
     'coordinates_end',
     'coordinates_strand']], how='left', left_on="gene", right_index=True)
annos_df['aspect'] = annos_df['term'].apply(lambda x: get_pd_row(terms_df, x)['aspect'])
annos_df['term'] = annos_df['term'].apply(lambda x: get_pd_row(terms_df, x))
annos_df['slim_terms'] = annos_df['slim_terms'].apply(lambda x: spread_terms(terms_df, x))
annos_df['qualifier'] = annos_df['qualifier'].str.replace('_', ' ')
annos_df['evidence'] = annos_df.apply(lambda x: get_evidence(articles_df, genes_df, x),axis=1)
annos_df['evidence_type'] = annos_df.apply(lambda x: get_evidence_type(x), axis=1)
annos_df['groups'] = annos_df['evidence'].apply(lambda x: get_groups(x))
annos_df['evidence_count'] = annos_df['evidence'].apply(lambda x: count_evidence(x))

   
anno_json = annos_df.to_json(orient="records", default_handler=None)
json_str = json.loads(anno_json)
#pprint.pprint(json_str)
write_to_json(json_str, ospath.join('.', clean_annotations_fp))
annos_df

NameError: name 'terms_df' is not defined

In [ ]:
annos_df['evidence_type'] = annos_df.apply(lambda x: get_evidence_type(x), axis=1)
annos_df[annos_df['evidence_type']=='direct']

In [ ]:
print(dict(annos_df.iloc[1]))

In [ ]:
clean_annos_df = pd.read_json(clean_annotations_fp)
clean_annos_df

In [ ]:
filtered_terms = terms_df[terms_df['is_goslim']==True]
filtered_terms

In [ ]:
temp_df = annos_df[annos_df.duplicated(subset=['gene', 'gene_symbol'])]
temp_df

In [ ]:
def add_terms():
    results = list()
    for n in range(0, len(human_df) -1):
        index = random.randrange(0, len(filtered_terms))
        results.append(dict(filtered_terms.iloc[index]))
    
    return results

count = 1

def add_gene():
    return human_df.iloc[count]['gene']

    
                            
#add_terms()

#ann2_df = pd.DataFrame.from_dict(add_terms())
ann2_df

ann3 = human_df.agg('-'.join, axis=1)
ann2_df

add_gene()
ann2_df['gene'] = ann2_df['id'].apply(lambda x: add_gene())
ann2_df

json_chunk = ann2_df.to_json(orient="records", default_handler=None)
json_str = json.loads(json_chunk)

write_to_json(json_str, 'out.json')
                     

In [9]:
clean_annos_df = pd.read_json(clean_annotations_fp)
clean_annos_df

,gene,gene_symbol,gene_name,term,slim_terms,qualifier,evidence,group,taxon_id,taxon_label,taxon_abbr,coordinates_chr_num,coordinates_start,coordinates_end,coordinates_strand,aspect,evidence_type,groups,evidence_count
0,UniProtKB:P17535,JUND,Transcription factor jun-D,"{'id': 'GO:0008134', 'label': 'transcription f...","[{'id': 'OTHER:0001', 'label': 'Other molecula...",None,"[{'with_gene_id': {'gene': 'UniProtKB:P05412',...",GO_Central,9606.0,Homo sapiens,Hsa,19,18279760.0,18280929.0,-1.0,molecular function,homology,"[UniProtKB, RGD, WB, MGI]",4
1,UniProtKB:P17535,JUND,Transcription factor jun-D,"{'id': 'GO:0000978', 'label': 'RNA polymerase ...","[{'id': 'GO:0003677', 'label': 'DNA binding', ...",None,"[{'with_gene_id': {'gene': 'UniProtKB:P05412',...",GO_Central,9606.0,Homo sapiens,Hsa,19,18279760.0,18280929.0,-1.0,molecular function,homology,"[UniProtKB, MGI]",3
2,UniProtKB:P17535,JUND,Transcription factor jun-D,"{'id': 'GO:0042127', 'label': 'regulation of c...","[{'id': 'OTHER:0002', 'label': 'Other biologic...",None,"[{'with_gene_id': {'gene': 'RGD:2943', 'gene_s...",GO_Central,9606.0,Homo sapiens,Hsa,19,18279760.0,18280929.0,-1.0,biological process,homology,"[MGI, UniProtKB, RGD]",3
3,UniProtKB:P17535,JUND,Transcription factor jun-D,"{'id': 'GO:0005667', 'label': 'transcription r...","[{'id': 'OTHER:0003', 'label': 'Other cellular...",None,"[{'with_gene_id': {'gene': 'UniProtKB:P17535',...",GO_Central,9606.0,Homo sapiens,Hsa,19,18279760.0,18280929.0,-1.0,cellular component,direct,"[MGI, UniProtKB, ZFIN, FB]",6
4,UniProtKB:P17535,JUND,Transcription factor jun-D,"{'id': 'GO:0006357', 'label': 'regulation of t...","[{'id': 'GO:0006355', 'label': 'regulation of ...",None,"[{'with_gene_id': {'gene': 'UniProtKB:P17535',...",GO_Central,9606.0,Homo sapiens,Hsa,19,18279760.0,18280929.0,-1.0,biological process,direct,"[UniProtKB, MGI, CGD, ZFIN, SGD, RGD, FB]",11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90901,UniProtKB:V9GZ38,,Uncharacterized protein (Fragment),"{'id': 'UNKNOWN:0002', 'label': 'Unknown biolo...","[{'id': 'UNKNOWN:0002', 'label': 'Unknown biol...",None,[],GO_Central,NaN,None,None,None,NaN,NaN,NaN,biological process,homology,[],0
90902,UniProtKB:V9GZ38,,Uncharacterized protein (Fragment),"{'id': 'UNKNOWN:0003', 'label': 'Unknown cellu...","[{'id': 'UNKNOWN:0003', 'label': 'Unknown cell...",None,[],GO_Central,NaN,None,None,None,NaN,NaN,NaN,cellular component,homology,[],0
90903,UniProtKB:X6R8D5,,Uncharacterized protein,"{'id': 'UNKNOWN:0001', 'label': 'Unknown molec...","[{'id': 'UNKNOWN:0001', 'label': 'Unknown mole...",None,[],GO_Central,NaN,None,None,None,NaN,NaN,NaN,molecular function,homology,[],0
90904,UniProtKB:X6R8D5,,Uncharacterized protein,"{'id': 'UNKNOWN:0002', 'label': 'Unknown biolo...","[{'id': 'UNKNOWN:0002', 'label': 'Unknown biol...",None,[],GO_Central,NaN,None,None,None,NaN,NaN,NaN,biological process,homology,[],0


In [10]:
class CustomJSONizer(json.JSONEncoder):
    def default(self, obj):
        return super().encode(bool(obj)) \
            if isinstance(obj, np.bool_) \
            else super().default(obj)
    
def count_unique_terms(annos_df):
    terms = set()
    for term in list(annos_df['term']):
        terms.add(term['id'])
    
    return list(terms)

#count_uniq = count_unique_terms()
#write_to_json(count_uniq, filtered_terms_fp, cls=CustomJSONizer)
#print(len(count_uniq))

def count_unique_slim_terms(annos_df):
    terms = set()
        
    for s_terms in list(annos_df['slim_terms']):
        for term in s_terms:
            terms.add(term['id'])
            
    return list(terms)

#count_uniq = count_unique_slim_terms()
#write_to_json(count_uniq, filtered_slim_terms_fp, cls=CustomJSONizer)
#print(count_uniq)
def count_unique_refs(annos_df):
    refs = set()
        
    for evidences in list(annos_df['evidence']):
        for evidence in evidences:
            for ref in evidence['references']:
                if ref is not None:
                    refs.add(ref['pmid'])
            
    return list(refs)

def count_unique_withs(annos_df):
    gene = set()
    gene_name=set()
        
    for evidences in list(annos_df['evidence']):
        for evidence in evidences:
            gene.add(evidence['with_gene_id']['gene'])
            
    return list(gene)

len(count_unique_refs(clean_annos_df))

56454

In [11]:
scalar_cols = {
    'gene', 
    'gene_symbol',
    'gene_name',
    'taxon_id', 
    'taxon_label', 
    'taxon_abbr', 
    'coordinates_chr_num', 
    'coordinates_start', 
    'coordinates_end', 
    'coordinates_strand', 
    'aspect', 
    'evidence_type'
}
    

stats = {k: len(clean_annos_df[k].unique()) for k in scalar_cols }

stats['terms']=len(count_unique_terms(clean_annos_df))
stats['slim_terms']=len(count_unique_slim_terms(clean_annos_df))
stats['references']=len(count_unique_refs(clean_annos_df))
stats['with_gene']=len(count_unique_withs(clean_annos_df))
stats_df = pd.DataFrame.from_dict(stats, orient ='index', columns=['Unique Count'])
stats_df


,Unique Count
gene_symbol,20541
coordinates_start,16446
evidence_type,2
taxon_label,2
gene_name,20460
coordinates_chr_num,26
aspect,3
taxon_id,2
taxon_abbr,2
coordinates_end,16432


In [12]:
unique_genes = clean_annos_df.drop_duplicates(subset=['gene', 'gene_symbol', 'gene_name'])
unique_genes = unique_genes[['gene', 'gene_symbol', 'gene_name']]
unique_genes = unique_genes[~unique_genes.duplicated(subset=['gene_symbol'])]
duplicate_genes = unique_genes[unique_genes.duplicated(subset=['gene'])]
duplicate_genes = duplicate_genes[['gene']]
res = unique_genes[unique_genes['gene'].isin(list(duplicate_genes['gene']))]
res = res.sort_values(by=['gene'])
res.to_csv(duplicate_gene_fp)
res

,gene,gene_symbol,gene_name
1750,UniProtKB:A0A1W2PRP0,A0A1W2PRP0,Uncharacterized protein
1755,UniProtKB:A0A1W2PRP0,,Uncharacterized protein
